In [1]:
import numpy as np
import pandas as pd
import pystan
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline

In [2]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(25,8); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

In [3]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

In [4]:
def load_data_stan(file_name, pk=1, edge=3, nk=5):
    with open(file_name, encoding='utf-8') as f: lines = f.read().split('\n')
    p = 0; playerid = {};
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid: playerid[player0]=p; p+=1
        if player1 not in playerid: playerid[player1]=p; p+=1

    nplayers = len(playerid)
    playername = ['']*nplayers
    for player in playerid:
        playername[ playerid[player] ]=player;  # id to name lookup


    pKeep = pk   # fraction of edges to consider (immed. throw out 1-p edges)
    nEdge = edge     # try to keep nEdge opponents per player (may be more; asymmetric)
    nKeep = nk     # keep at most nKeep games per opponent pairs (play each other multiple times)

    count = 0
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) );
    player_A = []
    player_B = []
    wins = []
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: continue;   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]];
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]';
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                count += 1
                nplays[a,b] += 1; nplays[b,a]+=1; nwins[a,b] += aw; nwins[b,a] += bw;
                player_A.append(a+1)
                player_B.append(b+1)
                wins.append(aw)

    win_re = []
    for i in wins:
        if i:
            win_re.append(1)
        else:
            win_re.append(0)
    return np.array([player_A,player_B,win_re]), playerid, playername

In [5]:
train_data, playerid, playername = load_data_stan("train.csv",1,1,10)
print(train_data.shape[1])

998


In [6]:
skill_data = {
    'N': len(playerid),
    'E': train_data.shape[1],
    'scale': 0.5,
    'win':train_data[2],
    'PA': train_data[0],
    'PB': train_data[1]
}

In [7]:
start = time.perf_counter()
fit = sm.sampling(data=skill_data, iter=200, chains=2)
print(time.perf_counter() - start)

4.3660988


# validate

In [8]:
def load_data(file_name):
    data=pd.read_csv(file_name,index_col=False,
                     names=['date', 'p1', 'p1_outcome', 'score', 'p2', 'p2_outcome', 'p1_race', 'p2_race', 'addon', 'type'])
    #drop other columns for now
    data.drop(columns=['date','score','p2_outcome','p1_race', 'p2_race', 'addon', 'type'], inplace=True)
    data['p1_outcome'].replace({"[loser]":0,"[winner]":1},inplace=True)
    return data

valid_data = load_data('valid.csv')
valid_data['p1'].replace(playerid,inplace=True)
valid_data['p2'].replace(playerid,inplace=True)
valid_games=[tuple((r[0],r[2],r[1])) for r in valid_data.to_numpy()]
#valid_games

In [9]:
def logit(z): return 1./(1.+np.exp(-z))
def validate(fits, valid_games, n_games):
    accuracy = []
    for fit in fits:
        samples = fits[fit].extract()
        acc, acc_simulate = 0, 0
        n = n_games
        #for g in valid_games[:n_games]:
        for g in [valid_games[i] for i in np.random.choice(len(valid_games), n, replace = False)]:
            try:
                i,j,result=int(g[0]),int(g[1]),int(g[2])
            except:
                n-=1
                continue

            prob = logit( skill_data['scale']*(samples['skill'][:,i]-samples['skill'][:,j]) ).mean()
            pred = 1 if prob > 0.5 else 0 #
            pred_simulate = np.random.choice([1,0], p=[prob, 1-prob])
            acc += (pred==result)
            acc_simulate += (pred_simulate==result)
        accuracy.append((acc/n, acc_simulate/n))
        print(fit, accuracy[-1], n)
    return accuracy

In [10]:
accuracy = validate({1:fit}, valid_games, 10000)
accuracy

1 (0.5513, 0.5412) 10000


[(0.5513, 0.5412)]

In [11]:
accuracy = validate({1:fit}, valid_games, len(valid_games))
accuracy

1 (0.5524375844352016, 0.547895369493761) 94007


[(0.5524375844352016, 0.547895369493761)]

# compare

In [12]:
def compare_nEdge(parameter_range, pKeep = 1, nKeep = 5, n_valid = 10000):
    result = {}
    for nEdge in parameter_range:
        train_data, playerid, playername = load_data_stan("train.csv", pKeep, nEdge, nKeep)
        print("Number of Training data:", train_data.shape[1])
        skill_data = {
            'N': len(playerid),
            'E': train_data.shape[1],
            'scale': 0.5,
            'win':train_data[2],
            'PA': train_data[0],
            'PB': train_data[1]
        }
        start = time.perf_counter()
        fit = sm.sampling(data=skill_data, iter=200, chains=2, control={'max_treedepth': 14})
        print("Training time:", time.perf_counter() - start)
        
        
        valid_data = load_data('valid.csv')
        valid_data['p1'].replace(playerid,inplace=True)
        valid_data['p2'].replace(playerid,inplace=True)
        valid_games=[tuple((r[0],r[2],r[1])) for r in valid_data.to_numpy()]
        accuracy = validate({nEdge:fit}, valid_games, n_valid)
        result[nEdge] = accuracy
    return result
        

In [13]:
compare_nEdge([2**i for i in range(8)])

Number of Training data: 998


Training time: 4.033962899999999
1 (0.5561, 0.5449) 10000
Number of Training data: 2808


Training time: 13.208783699999998
2 (0.5976, 0.5679) 10000
Number of Training data: 6608
Training time: 30.952792000000002
4 (0.6245, 0.5797) 10000
Number of Training data: 14370
Training time: 65.75949179999999
8 (0.6498, 0.5951) 10000
Number of Training data: 29739
Training time: 142.90399879999998
16 (0.6734, 0.5939) 10000
Number of Training data: 60596


Training time: 92.0408066
32 (0.6861, 0.5991) 10000
Number of Training data: 102716


Training time: 400.4612805
64 (0.6906, 0.6086) 10000
Number of Training data: 127579


Training time: 243.68269900000007
128 (0.6836, 0.6042) 10000


{1: [(0.5561, 0.5449)],
 2: [(0.5976, 0.5679)],
 4: [(0.6245, 0.5797)],
 8: [(0.6498, 0.5951)],
 16: [(0.6734, 0.5939)],
 32: [(0.6861, 0.5991)],
 64: [(0.6906, 0.6086)],
 128: [(0.6836, 0.6042)]}

In [16]:
compare_nEdge(range(20,81,10))

Number of Training data: 37448


Training time: 80.4659153
20 (0.6762, 0.6) 10000
Number of Training data: 56788


Training time: 85.91289299999994
30 (0.6835, 0.6003) 10000
Number of Training data: 74512


Training time: 112.49282820000008
40 (0.6819, 0.6056) 10000
Number of Training data: 88935


Training time: 138.56850680000002
50 (0.684, 0.6032) 10000
Number of Training data: 99435


Training time: 170.40448319999996
60 (0.6889, 0.6021) 10000
Number of Training data: 107207


Training time: 189.02928829999973
70 (0.6888, 0.6093) 10000
Number of Training data: 113210


Training time: 205.08381740000004
80 (0.6836, 0.6033) 10000


{20: [(0.6762, 0.6)],
 30: [(0.6835, 0.6003)],
 40: [(0.6819, 0.6056)],
 50: [(0.684, 0.6032)],
 60: [(0.6889, 0.6021)],
 70: [(0.6888, 0.6093)],
 80: [(0.6836, 0.6033)]}